# Keras

Keras 는 딥 러닝을 모델을 만들고 학습하기 위한 High level API이다. 

## Import tf.keras

`tf.keras` 는 Keras API 의 Tensorflow 구현체이며, `eager execution`, `tf.data` pipeline, `Estimater` 같은 TensorFlow 기능들을 지원한다. 

In [3]:
import tensorflow as tf
from tensorflow import keras

- 최신 tensorFlow의 `tf.keras`의 버전은 pyPI의 최신 버전 keras와는 차이가 있다. 
- `tf.keras` 에서는 기본적으로 모델 저장 시에 checkpoint format을 사용하고, HDF5 포멧을 사용한다.

## Build a simple model

### Sequential model

Keras에서는 모델을 생성할때 layer를 조립하게 되며, 대부분의 model의 type은 layer의 stack형태로 구성된다. 
`tf.keras.Sequential`

In [2]:
model = keras.Sequential()

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

### Configure the layers

`tf.keras.layers`에는 다음과 같은 contructor parameters 가 있다.

- `activation` : layer의 activation function을 설정한다. 
- `kernel_initializer`, `bias_initializer` : layer의 weight 와 bias의 초기화 scheme을 설정한다. 
- `kernel_regularizer`, `bias_regularizer` : layer의 weight나 bias에 적용할 정규화 scheme을 설정한다. 

In [4]:
from tensorflow.keras import layers

# Create a sigmoid layer:
layers.Dense(64, activation='sigmoid')
# Or:
layers.Dense(64, activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.01))
# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
layers.Dense(64, bias_regularizer=keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')
# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=keras.initializers.constant(2.0))

## Train and evaluate

### Set up training

모델을 구축한 뒤에, compile 메소드를 호출하여 학습 과정을 설정해야 한다.

In [5]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [6]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

### Input Numpy data

In [8]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

#model.fit(data, labels, epochs=10, batch_size=32)

### Input tf.data datasets

`Datasets API`를 사용하면 large dataset이나 multi-device training으로 scaling 할 수 있다. 

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30)

`fit` 에서 설정하는 `steps_per_epoch` 는 다음 epoch 으로 이동하기 이전에 모델이 실행되는 학습 Step의 수이다. 

### Evaluate and predict 

`tf.keras.Model.evaluate` 와  `tf.keras.Model.predict` 는 Numpy Data와 tf.data.Dataset을 사용할 수 있다.

model.evaluate(x, y, batch_size=32) <br>
model.evaluate(dataset, steps=30)

## Build advanced models

### Functional API

`tf.keras.Sequential` model 은 단순한 Layer stack 만 지원하므로, arbitrary model을 표현할 수 없다. 다음과 같은 복잡한 Model topologies를 만들기 위해서는 **Keras functional API**를 사용해야 한다.

- Multi-input models
- Multi-output models,
- Models with shared layers  (동일한 Layer 반복 사용)
- Models with non-sequential data flow (residual connections)

functional API를 사용하여 Model을 만드는 과정은 다음과 같다. 

1. Layer instance는 호출가능하며, tensor를 리턴한다.
2. Input tensors와 output tensors는 `tf.keras.Model` instance를 정의하는데 사용된다. 
3. 이 모델은 `Sequential` model 과 유사하게 학습된다.

In [1]:
# build fully-connected network using functional API
input = keras.Input(shape =(32, ))

x = keras.layers.Dense(64, activation='relu')(input)
x = keras.layers.Dense(64, activation='relu')(x)

predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

NameError: name 'keras' is not defined

### Model subclassing

`tf.keras.Model`을 subclassing 하여 full-customizable model을 만들어 보자. 

`__init__` method 내부에서 layer를 생성하고, class instance의 attributes를 설정하자. 그리고 `call` method 내부에서 forward pass 를 정의한다. 

Model subclassing은 forward pass 가 imperatively 하게 구현할 수 있으므로 eager execution을 사용할때 유용하다. 

In [ ]:
# subclassing example 
class MyModel(keras.Model):
    
    def __init__(self, num_classess=10):
        super(MyModel, self).__init__(name = 'my_model')
        self.num_classes = num_classes
        
        self.dense_1 = keras.layers.Dense(32, activation='relu')
        self.dense_2 = keras.layers.Dense(num_classes, activation='sigmoid')
    
    def call(self, inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)
    
model = MyModel(num_classe = 10)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

### Custom Layers

`tf.kears.layers.Layer` subclassing을 통해서 custom layer를 생성할 때, 다음의 method를 구현해야 한다.

- build : layer의 weight를 생성한다. `add_weight` method를 사용해서 weight를 추가한다.
- call : forward pass를 정의한다.
- compute_output_shape : 주어진 input shape 에 대해서 Layer의 output shape을 어떻게 계산할건지 정의한다. 
- 추가적으로 `get_config` method와 `from_config` method를 구현하여 layer를 serialize 할 수 있다. 

In [2]:
# Custom layer example 
class MyLayer(keras.layers.Layer):
    
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        
        self.kernel = self.add_weight(name='kernel',
                                     shape=shape,
                                     initializer='uniform',
                                     trainable=True)
        super(MyLayer, self).build(input_shape)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        
    @classmethod
    def from_config(cls, config):
        return cls(**config)
    

model = keras.Sequential([MyLayer(10),
                         keras.layers.Activation('softmax')])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(data, targets, batch_size=32, epochs=5)

NameError: name 'keras' is not defined

## Save and restore

### Weights only

`tf.keras.Model.save_weights`를 사용하면 model의 weight를 저장할 수 있다.

In [ ]:
model.save_weights('./my_model')

model.load_weights('my_model')

기본적으로 이 방법은 model의 weights를 TensorFlow checkpoint file format으로 저장한다. Weights는 Keras HDF5 format으로도 저장할 수 있다. 

In [ ]:
model.save_weights('my_model.h5', save_format='h5')
model_load_weights('my_model.h5')

### Configuration only

Model의 configuration 저장은 weights를 제외한 model architecture를 serialization 해서 저장하는 것이다. 저장된 cofiguration은 동일한 모델을 다시 생성할 수 있고, 초기화 할 수 있다. Keras는 JSON과 YAML serialization format을 지원한다.

In [ ]:
json_string = model.to_json()

fresh_model = keras.models.from_json(json_string)

yaml_string = model.to_yaml()

fresh_model = keras.models.from_yaml(yaml_string)

### Entire model

weight values, model's configuration, optimizer's configuration을 모두 포함한 파일로 전체 모델을 저장할 수 있다. model의 checkpoint 형태로 저장할 수 있으며, 차후 학습을 이어서 할 수 있다. 

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(10, activation='softmax', input_shape=(32,)),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metircs=['accuracy'])
model.fit(data, target, batch_size=32, epochs=5)

model.save('my_model.h5')

model = keras.models.load_model('my_model.h5')

## Distribution 

### Estimator 

`Estimator` API는 배포 환경을 위한 모델을 학습하는데 사용한다. 특히 큰 데이터를 분산 학습하고, production을 위한 모델을 배포하는 경우를 타겟으로 하고 있다. 

`tf.keras.Model` 은 `tf.estimator` API를 통해서 학습할 수 있으며, model은 `tf.keras.estimator.model_to_estimator`를 통해서 `tf.estimator.Estimator`object로 변환된다. 

In [ ]:
model = keras.Sequential([layers.Dense(10, activation='softmax'),
                         layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

estimator = keras.estimator.model_to_estimator(model)

### Multiple GPU

`tf.keras` model 은 `tf.contrib.distribute.DistributionStrategy`를 사용하여 multiple GPU에서 동작한다. 이 API는 기존 코드에 대한 거의 변경 없이 multiple GPU에서의 분산 학습을 가능하게 해준다. 

`DistributionStrategy`를 keras에서 사용하기 위해서는 `tf.keras.Model`을 `tf.estimator.Estimator`로 변환 하고, estimator를 학습 시킨다. 

다음은 single machine에서 multiple GPU를 사용하는 방법에 대한 예제이다. 


먼저, 간단한 모델을 정의하자.

In [5]:
model = keras.Sequential()
model.add(keras.layers.Dense(16, activation='relu', input_shape=(10, )))
model.add(keras.layers.Dense(1, activation='sigmoid'))

optimizer = tf.train.GradientDescentOptimizer(0.2)

model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 16)                176       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


이어서 *input pipeline* 을 정의한다. `input_fn` 은 multiple device 간에 data를 분산 시키는 `tf.data.Dataset` object 를 리턴한다.  각 device는 input batch의 slice 를 처리한다. 

In [6]:
def input_fn():
    x = np.random.random((1024, 10))
    y = np.random.randint(2, size=(1024, 1))
    x = tf.cast(x, tf.float32)
    dataset = tf.data.Dataset.from_tensor_slices((x, y)) # Tensor를 slice 단위로 변경 
    dataset = dataset.repeat(10)
    dataset = dataset.batch(32)
    return dataset

그다음 `tf.estimator.RunConfig` 를 생성한다. 그리고 `tf.contrib.distribute.MirroredStrategy` instance 에 `train_distribute` argument를 설정한다. `MirrorStrategy` 를 생성할 때, device list를 정의하거나 `num_gpus` argument를 설정할 수 있다. 기본적으로는 모든 사용가능한 GPU를 사용하도록 되어 있다.

In [7]:
strategy = tf.contrib.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

그리고 keras model을 `tf.estimator.Estimator` instance로 변환한다. 

In [8]:
keras_estimator = keras.estimator.model_to_estimator(
    keras_model = model,
    config = config,
    model_dir = '/tmp/model_dir'
)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7fcbae0b8dd8>, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcbae0b8f98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


마지막으로, `Estimator` instance 를 학습하